In [18]:
# Access voter_data from 
%store -r voter_data

In [19]:
voter_data

,Voters_VotingPerformanceEvenYearGeneral_%,Voters_VotingPerformanceEvenYearPrimary_%,CommercialData_EstimatedHHIncomeAmount_$,CommercialData_EstHomeValue_$,County,Ethnic_Description,Unified_School_District
0,50.0,100.0,50.0,100.0,ALBANY,Irish,ALBANY CNTY SD 1 (EST.)
1,14.0,14.0,14.0,14.0,ALBANY,English/Welsh,ALBANY CNTY SD 1 (EST.)
2,100.0,100.0,100.0,100.0,ALBANY,Dutch (Netherlands),Unknown
3,100.0,50.0,100.0,50.0,ALBANY,Irish,ALBANY CNTY SD 1 (EST.)
4,33.0,20.0,33.0,20.0,ALBANY,English/Welsh,ALBANY CNTY SD 1 (EST.)
...,...,...,...,...,...,...,...
290403,0.0,50.0,0.0,50.0,WESTON,English/Welsh,Unknown
290404,33.0,0.0,33.0,0.0,WESTON,German,WESTON CNTY SD 1 (EST.)
290405,85.0,57.0,85.0,57.0,WESTON,Unknown,WESTON CNTY SD 1 (EST.)
290406,66.0,0.0,66.0,0.0,WESTON,English/Welsh,WESTON CNTY SD 1 (EST.)


In [29]:
## Anshul's version of one hot encoding

voter_data.ethnic

pandas.core.frame.DataFrame

In [20]:
# SVM

#threshold = 0.646 
# Reference for threshold value from: https://www.sos.state.mn.us/media/4446/us-turnout-map-2020.pdf

# 1: float
# 2: float
# 3: float
# 4: float
# 5: str
# 6: str
# 7: str
# 8: str

In [26]:
# OneHot Encoding for Modeling

import pandas as pd
from sklearn.compose import ColumnTransformer

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# List of categorical columns to be one-hot encoded
categorical_columns = ['Ethnic_Description', 'County', 'Unified_School_District']

# Create a column transformer that will apply the encoder to the specified columns
column_transformer = ColumnTransformer(transformers=[
    ('encoder', encoder, categorical_columns)],
    remainder='passthrough'  # This parameter specifies to keep the remaining columns untouched
)

# Fit and transform the data, applying one-hot encoding to the specified columns
voter_data_encoded_array = column_transformer.fit_transform(voter_data)

# The output is a NumPy array, convert it back to dataframe
new_columns = column_transformer.get_feature_names_out()


voter_data_encoded = pd.DataFrame(voter_data_encoded_array, columns=new_columns)

# Ensure the index aligns with the original DataFrame, if necessary
voter_data_encoded.index = voter_data.index
print(voter_data_encoded)

        encoder__Ethnic_Description_Afghan  \
0                                      0.0   
1                                      0.0   
2                                      0.0   
3                                      0.0   
4                                      0.0   
...                                    ...   
290403                                 0.0   
290404                                 0.0   
290405                                 0.0   
290406                                 0.0   
290407                                 0.0   

        encoder__Ethnic_Description_Albanian  \
0                                        0.0   
1                                        0.0   
2                                        0.0   
3                                        0.0   
4                                        0.0   
...                                      ...   
290403                                   0.0   
290404                                   0.0   
290405         

In [22]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline

float_columns = ['Voters_VotingPerformanceEvenYearGeneral_%', 'Voters_VotingPerformanceEvenYearPrimary_%', 'CommercialData_EstimatedHHIncomeAmount_$', 'CommercialData_EstHomeValue_$']
categorical_columns = ['County', 'Ethnic_Description', 'Unified_School_District']

# Define the ColumnTransformer, specifying one-hot encoding for categorical variables
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_columns)
    ],
    remainder='passthrough'  
)


In [24]:
# SPLIT DATA

'''

PAUSED HERE, read comments below

'''

# See which of the existing columns can count as voter turnout? maybe Voters_VotingPerformanceEvenYearGeneral_% or Voters_VotingPerformanceEvenYearPrimary_%?
# This column should represent the prediction column, replacing 'target'

voter_data_copy = voter_data

y = voter_data_copy['Voters_VotingPerformanceEvenYearGeneral_%']  # Target variable
X = voter_data_copy.drop('Voters_VotingPerformanceEvenYearGeneral_%', axis=1)  # Feature matrix

print(X)

'''
random state=42, check if this is correct, prolly needa fix this
'''

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# TRAIN SVM

# Create the SVM model pipeline
svm_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', SVC(kernel='linear'))])

# Train the SVM model on the training set
svm_pipeline.fit(X_train, y_train)

# Optionally, evaluate the model on the test set
from sklearn.metrics import accuracy_score

y_pred = svm_pipeline.predict(X_test)
print(f"Test Accuracy: {accuracy_score(y_test, y_pred)}")



        Voters_VotingPerformanceEvenYearPrimary_%  \
0                                           100.0   
1                                            14.0   
2                                           100.0   
3                                            50.0   
4                                            20.0   
...                                           ...   
290403                                       50.0   
290404                                        0.0   
290405                                       57.0   
290406                                        0.0   
290407                                       50.0   

        CommercialData_EstimatedHHIncomeAmount_$  \
0                                           50.0   
1                                           14.0   
2                                          100.0   
3                                          100.0   
4                                           33.0   
...                                          ...   

ValueError: Found unknown categories ['Uzbek'] in column 1 during transform

In [30]:
voter_data_encoded

,encoder__Ethnic_Description_Afghan,encoder__Ethnic_Description_Albanian,encoder__Ethnic_Description_Arab,encoder__Ethnic_Description_Armenian,encoder__Ethnic_Description_Australian,encoder__Ethnic_Description_Austrian,encoder__Ethnic_Description_Azerb,encoder__Ethnic_Description_Belgian,encoder__Ethnic_Description_Bhutanese,encoder__Ethnic_Description_Bosnian Muslim,...,encoder__Unified_School_District_UINTA CNTY SD 6 (EST.),encoder__Unified_School_District_Unknown,encoder__Unified_School_District_WASHAKIE CNTY SD 1 (EST.),encoder__Unified_School_District_WASHAKIE CNTY SD 2 (EST.),encoder__Unified_School_District_WESTON CNTY SD 1 (EST.),encoder__Unified_School_District_WESTON CNTY SD 7 (EST.),remainder__Voters_VotingPerformanceEvenYearGeneral_%,remainder__Voters_VotingPerformanceEvenYearPrimary_%,remainder__CommercialData_EstimatedHHIncomeAmount_$,remainder__CommercialData_EstHomeValue_$
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,50.0,100.0,50.0,100.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,14.0,14.0,14.0,14.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,100.0,100.0,100.0,100.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,100.0,50.0,100.0,50.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,33.0,20.0,33.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290403,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,50.0,0.0,50.0
290404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,33.0,0.0,33.0,0.0
290405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,85.0,57.0,85.0,57.0
290406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,66.0,0.0,66.0,0.0


In [ ]:
# DATA CLEANING

import pandas as pd

# remove dollar signs
voter_data['CommercialData_EstHomeValue'] = voter_data['CommercialData_EstHomeValue'].str[1:]
voter_data['CommercialData_EstimatedHHIncomeAmount'] = voter_data['CommercialData_EstimatedHHIncomeAmount'].str[1:]

# remove percent signs
voter_data['Voters_VotingPerformanceEvenYearGeneral'] = voter_data['Voters_VotingPerformanceEvenYearGeneral'].str[:-1]
#voter_data['Voters_VotingPerformanceEvenYearPrimary'] = voter_data['Voters_VotingPerformanceEvenYearPrimary'].str[:-1]


# convert non-numeric strings to NaN
voter_data['CommercialData_EstHomeValue'] = pd.to_numeric(voter_data['CommercialData_EstHomeValue'], errors='coerce')
voter_data['CommercialData_EstimatedHHIncomeAmount'] = pd.to_numeric(voter_data['CommercialData_EstimatedHHIncomeAmount'], errors='coerce')
voter_data['Voters_VotingPerformanceEvenYearGeneral'] = pd.to_numeric(voter_data['Voters_VotingPerformanceEvenYearGeneral'], errors='coerce')
#voter_data['Voters_VotingPerformanceEvenYearPrimary'] = pd.to_numeric(voter_data['Voters_VotingPerformanceEvenYearPrimary'], errors='coerce')

voter_data = voter_data.rename(columns={
    'CommercialData_EstHomeValue': 'CommercialData_EstHomeValue_$',
    'CommercialData_EstimatedHHIncomeAmount': 'CommercialData_EstimatedHHIncomeAmount_$',
    'Voters_VotingPerformanceEvenYearGeneral' : 'Voters_VotingPerformanceEvenYearGeneral_%',
    #'Voters_VotingPerformanceEvenYearPrimary' : 'Voters_VotingPerformanceEvenYearPrimary_%'
    
})

# Replace 'None' values with 'Unknown' in 'Ethnic_Description' and 'Unified_School_District' column
voter_data['Ethnic_Description'] = voter_data['Ethnic_Description'].replace([None, 'None'], 'Unknown')
voter_data['County'] = voter_data['County'].replace([None, 'None'], 'Unknown')
voter_data['Voters_Gender'] = voter_data['Voters_Gender'].replace([None, 'None'], 'Unknown')

'''Median value is used to impute Voters_VotingPerformanceEvenYearGeneral_%, Voters_VotingPerformanceEvenYearPrimary_%, 
CommercialData_EstimatedHHIncomeAmount_$, and CommercialData_EstHomeValue_$ columns '''

# Calculate the median for the numeric variable columns
column1_med = voter_data['Voters_VotingPerformanceEvenYearGeneral_%'].median()
#column2_med = voter_data['Voters_VotingPerformanceEvenYearPrimary_%'].median()
column3_med = voter_data['CommercialData_EstimatedHHIncomeAmount_$'].median()
column4_med = voter_data['CommercialData_EstHomeValue_$'].median()



# Replace NaN values with the corresponding median
voter_data['Voters_VotingPerformanceEvenYearGeneral_%'] = voter_data['Voters_VotingPerformanceEvenYearGeneral_%'].fillna(column1_med)
#voter_data['Voters_VotingPerformanceEvenYearPrimary_%'] = voter_data['Voters_VotingPerformanceEvenYearPrimary_%'].fillna(column2_med)
voter_data['CommercialData_EstimatedHHIncomeAmount_$'] = voter_data['CommercialData_EstimatedHHIncomeAmount_$'].fillna(column3_med)
voter_data['CommercialData_EstHomeValue_$'] = voter_data['CommercialData_EstHomeValue_$'].fillna(column4_med)
#voter_data['CommercialData_EstHomeValue_$'] = voter_data['Voters_VotingPerformanceEvenYearPrimary_%'].fillna(column2_med)


voter_data


In [4]:
# GRADIENT BOOSTING

# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score

# Assuming your data is stored in a DataFrame called 'voter_data'
# Provide a target variable for the regression task
voter_data['Voters_VotingPerformanceEvenYearGeneral'] = 'Voters_VotingPerformanceEvenYearGeneral'# Your target variable here

# Splitting data into features and target variable
X = voter_data.drop('Voters_VotingPerformanceEvenYearGeneral', axis=1)  # Features
y = voter_data['Voters_VotingPerformanceEvenYearGeneral']  # Target variable

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define numeric features and categorical features
numeric_features = ['CommercialData_EstimatedHHIncomeAmount', 'CommercialData_EstHomeValue']
categorical_features = ['County', 'Ethnic_Description', 'Voters_Gender']

# Define preprocessing steps
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine preprocessing steps for numeric and categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Define the model
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', GradientBoostingRegressor())])

# Train the model
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate the model
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R^2 Score:", r2_score(y_test, y_pred))


NameError: name 'voter_data' is not defined

## DNN/CNN

In [ ]:
!pip install torch torchvision

In [ ]:
import torch
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LinearSVC
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Assemble the feature vector
feature_columns = ['HH_Income_Amount','Home_Value','Voters_Male',
                   'CountyIndex','EthnicIndex','CountyVec','EthnicVec']
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
voter_data_assembled = assembler.transform(voter_data_encoded)

# Split the data into training and test sets
train_data = voter_data_assembled.sample(withReplacement=False, fraction=0.8, seed=234)
test_data = voter_data_assembled.subtract(train_data)

# Extract features (X) and target label (Y) columns from train_data
X_train = train_data.select('features').rdd.map(lambda x: x[0]).collect()
Y_train = train_data.select('Voters_Active').rdd.map(lambda x: x[0]).collect()

# Extract features (X) and target label (Y) columns from test_data
X_test = test_data.select('features').rdd.map(lambda x: x[0]).collect()
Y_test = test_data.select('Voters_Active').rdd.map(lambda x: x[0]).collect()

# Convert features (X) and target label (Y) to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32)



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Define the DNN model architecture
class SimpleDNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleDNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

    
input_size = 95
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 10
batch_size = 64

# Define the DNN model
model = SimpleDNN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
# Create DataLoader for training data
#train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
#train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#print("Shape of X_train_tensor:", X_train_tensor.shape)
#print("Shape of Y_train_tensor:", Y_train_tensor.shape)

#voter_data_encoded.filter(voter_data_encoded['Voters_Active'].isNull()).count()


# len(X_train)# 229,995
# len(Y_train) # 229,688

#voter_data_encoded.count() # 287036

# len(voter_data_encoded.columns) #8


0.8*287036

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Define the DNN model architecture
class SimpleDNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleDNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

# Define hyperparameters
input_size = 95
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 10
batch_size = 64

# Define the DNN model
model = SimpleDNN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create DataLoader for training data
train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Training loop
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels.view(-1, 1))  # BCELoss expects a 2D tensor for binary classification
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Print loss every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Convert model to evaluation mode
model.eval()

# Make predictions on test data
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = (outputs >= 0.5).float()  # Convert probabilities to binary predictions (0 or 1)
    accuracy = (predictions == Y_test_tensor.view(-1, 1)).float().mean()
    print('Test Accuracy:', accuracy.item())


In [ ]:
# Check the shape of input tensors
print("X_train_tensor shape:", X_train_tensor.shape)
print("X_test_tensor shape:", X_test_tensor.shape)

# Verify input size matches the number of features
input_size = X_train_tensor.shape[1]  # Update input_size based on the actual number of features
